In [1]:
import pandas as pd
import numpy as np
import tensorflow

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

seed = 2000
np.random.seed(seed)
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Embedding
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence

from gensim.models import Word2Vec
from gensim.parsing.porter import PorterStemmer

import nltk
from nltk.tokenize import TweetTokenizer

import warnings
warnings.filterwarnings("ignore")

2023-01-04 15:32:26.293736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_sub1_df = pd.read_csv('/Users/nitanshjain/Documents/Miscellaneous/SemEval/semeval2023task3/preprocessed_data/subtask1/train_subtask_1.csv')
print(train_sub1_df.shape)
train_sub1_df.head()
print(train_sub1_df.genre.value_counts())

(433, 10)
opinion      382
reporting     41
satire        10
Name: genre, dtype: int64


In [3]:
le = LabelEncoder()
train_sub1_df['genre'] = le.fit_transform(train_sub1_df['genre'])
print(train_sub1_df.genre.value_counts())
train_sub1_df.head()

0    382
1     41
2     10
Name: genre, dtype: int64


,id,genre,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,833042063,2,Chelsea Handler Admits She’s ‘Very Sexually At...,Far-left comedienne Chelsea Handler has admitt...,chelsea handler admit she s very sexually attr...,"[(Chelsea, 'NNP'), (Handler, 'NNP'), (admit, '...","[(Chelsea Handler, 'PERSON', 380), (Robert Mue...",far leave comedienne chelsea handler have admi...,"[(far, 'RB'), (leave, 'VB'), (comedienne, 'NNP...","[(Chelsea Handler, 'PERSON', 380), (FBI Specia..."
1,832959523,2,How Theresa May Botched\n,Those were the times…\nThe Times page 1 is of ...,how theresa may botch,"[(how, 'WRB'), (Theresa, 'NNP'), (May, 'NNP'),...","[(Theresa, 'GPE', 384), (May, 'DATE', 391)]",those be the time the times page 1 be of janua...,"[(those, 'DT'), (be, 'VBP'), (the, 'DT'), (tim...","[(Times, 'ORG', 383), (1, 'CARDINAL', 397), (J..."
2,833039623,2,Robert Mueller III Rests His Case—Dems NEVER W...,Carload of crazies headed to the White House w...,robert mueller iii rest his case dems never will,"[(Robert, 'NNP'), (Mueller, 'NNP'), (III, 'NNP...","[(Robert Mueller III, 'PERSON', 380), (Dems, '...",carload of crazy head to the white house want ...,"[(Carload, 'NNP'), (of, 'IN'), (crazy, 'JJ'), ...","[(the White House, 'ORG', 383), (Barack Obama,..."
3,833032367,2,Robert Mueller Not Recommending Any More Indic...,"But of course, this makes no difference to the...",robert mueller not recommend any more indictment,"[(Robert, 'NNP'), (Mueller, 'NNP'), (not, 'RB'...","[(Robert Mueller, 'PERSON', 380)]",but of course this make no difference to the p...,"[(but, 'CC'), (of, 'IN'), (course, 'NN'), (thi...","[(the New York Times, 'ORG', 383), (late Frida..."
4,814777937,2,The Far Right Is Trying to Co-opt the Yellow V...,"This weekend in Manchester, England, Yellow Ve...",the far right be try to co opt the yellow vests,"[(the, 'DT'), (Far, 'NNP'), (right, 'NN'), (be...",[],this weekend in manchester england yellow vest...,"[(this, 'DT'), (weekend, 'NN'), (in, 'IN'), (M...","[(this weekend, 'DATE', 391), (Manchester, 'GP..."


In [4]:
def creating_tokens(df):
    tokens = list()
    tokenizer = TweetTokenizer()
    
    for tweets in df.loc[:, 'preprocessed_headlines']:
        # print(len(tokenizer.tokenize(tweets)))
        tokens.append(tokenizer.tokenize(tweets))
    
    df['tokens_headlines'] = tokens
    
    porter_stemmer = PorterStemmer()
    # Get the stemmed_tokens
    df['stemmed_tokens_headlines'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in df['tokens_headlines']]
    
    return df

In [5]:
train_sub1_df = creating_tokens(train_sub1_df)
x_tokens = train_sub1_df.stemmed_tokens_headlines
x_tokens = x_tokens.to_frame().reset_index()

y = train_sub1_df.genre

OUTPUT_FOLDER = '/Users/nitanshjain/Documents/Miscellaneous/SemEval/semeval2023task3/codefiles/subtask1/word2vec/cbow/headlines/'

tokens = pd.Series(train_sub1_df.stemmed_tokens_headlines).values
# print(tokens)
word2vec_model_file = OUTPUT_FOLDER + 'word2vec_subtask1_' + str(200) + '.model'

w2v_model = Word2Vec(tokens, min_count=1, vector_size=200, window=5, workers=4, sg=0)
w2v_model_lstm.train(tokens, epochs=10, total_examples=len(tokens))
w2v_model_lstm.save(word2vec_model_file)

In [6]:
def create_file(create_file, model_file, x):
    sg_w2v_model = Word2Vec.load(model_file)
    
    with open(create_file, 'w+') as word2vec_file:
        for index, row in x.iterrows():
            model_vector = (np.mean([sg_w2v_model_lstm.wv[token] for token in row['stemmed_tokens_headlines']], axis=0)).tolist()
            if index == 0:
                header = ",".join(str(ele) for ele in range(200))
                word2vec_file.write(header)
                word2vec_file.write("\n")
            
            if type(model_vector) is list:
                line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            else:
                line1 = ",".join([str(0) for i in range(200)])
            word2vec_file.write(line1)
            word2vec_file.write('\n')
    
    df = pd.read_csv(create_file)
    return df
        

In [7]:
word2vec_train_filename = OUTPUT_FOLDER + 'word2vec_subtask1_train_' + str(200) + '.csv'
word2vec_train_df = create_file(word2vec_train_filename, word2vec_model_file, x_tokens)
print(word2vec_train_df.shape)
word2vec_train_df.head()

(433, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.007085,-0.008194,-0.002214,0.017362,0.040222,-0.040511,0.016618,0.077604,-0.045723,0.028144,...,0.048570,-0.012475,-0.042139,-0.036501,0.045477,0.015220,0.023085,-0.054435,-0.005193,-0.012724
1,0.005563,-0.003317,-0.001562,0.004687,0.015490,-0.014568,0.005605,0.030678,-0.015025,0.008697,...,0.020343,-0.003145,-0.015172,-0.011079,0.015255,0.005743,0.008653,-0.020379,-0.001568,-0.002600
2,0.006431,-0.007063,-0.002085,0.014969,0.031369,-0.035100,0.013000,0.062723,-0.036408,0.021888,...,0.040054,-0.010619,-0.033625,-0.032450,0.038797,0.012208,0.017354,-0.043238,-0.005767,-0.010476
3,0.007527,-0.008113,-0.002385,0.016237,0.034489,-0.034324,0.014282,0.068088,-0.038283,0.024358,...,0.041918,-0.010143,-0.037023,-0.034411,0.039495,0.012512,0.019772,-0.047945,-0.005597,-0.007278
4,0.011333,-0.011088,-0.004654,0.022425,0.052368,-0.053078,0.021834,0.103699,-0.061094,0.036808,...,0.066037,-0.014187,-0.055245,-0.050965,0.061175,0.021086,0.030504,-0.073436,-0.009847,-0.014801


In [8]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

x_scaled = pd.DataFrame(mm.fit_transform(word2vec_train_df))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)
x_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.383416,0.458594,0.445066,0.468628,0.490771,0.552028,0.442867,0.457312,0.551524,0.429159,...,0.428392,0.540870,0.539993,0.548413,0.456160,0.471306,0.438478,0.545348,0.675850,0.419289
1,0.298704,0.717445,0.513186,0.137072,0.183481,0.859357,0.122241,0.167943,0.878236,0.094281,...,0.150422,0.863890,0.860465,0.863237,0.133860,0.175254,0.142900,0.845785,0.879151,0.848187
2,0.347046,0.518640,0.458544,0.406029,0.380773,0.616124,0.337527,0.365550,0.650664,0.321420,...,0.344528,0.605118,0.641177,0.598576,0.384930,0.377220,0.321091,0.644127,0.643616,0.514533
3,0.408014,0.462904,0.427146,0.439212,0.419543,0.625317,0.374846,0.398635,0.630704,0.363957,...,0.362892,0.621598,0.600798,0.574293,0.392373,0.386717,0.370621,0.602599,0.653157,0.649990
4,0.619903,0.305027,0.189707,0.601084,0.641677,0.403159,0.594710,0.618227,0.387943,0.578353,...,0.600408,0.481579,0.384247,0.369280,0.623568,0.654567,0.590413,0.377727,0.414763,0.331278


# KNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn_params = {
    'n_neighbors':range(1,15),
    'weights':['uniform', 'distance']
}

knn = KNeighborsClassifier()
clf = GridSearchCV(knn, knn_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8822410147991546
Best Parameters {'n_neighbors': 7, 'weights': 'uniform'}


# Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
lr_params = {
    'C':np.logspace(-2,2,7), 
    'penalty':['l1', 'l2', 'elasticnet', None]
}

lr = LogisticRegression()
clf = GridSearchCV(lr, lr_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8829985905567305
Best Parameters {'C': 4.6415888336127775, 'penalty': 'l2'}


# Decison Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier

dt_params = {
    'max_depth':range(2,15),
    'criterion':['gini', 'entropy', 'log_loss'],
    'splitter':['best', 'random']
}

dt = DecisionTreeClassifier()
clf = GridSearchCV(dt, dt_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8799330514446796
Best Parameters {'criterion': 'gini', 'max_depth': 2, 'splitter': 'random'}


# Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier

rfc_params = {
    'max_depth':range(3,15),
    'criterion':['gini', 'entropy', 'log_loss'],
}

rfc = RandomForestClassifier()
clf = GridSearchCV(rfc, rfc_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8822410147991546
Best Parameters {'criterion': 'gini', 'max_depth': 3}


# Multinomial Naive Bayes

In [13]:
%%time
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(x_scaled, y)

print('Train Accuracy Score', mnb.score(x_scaled, y))

Train Accuracy Score 0.8683602771362586
CPU times: user 10.2 ms, sys: 8.1 ms, total: 18.3 ms
Wall time: 17.2 ms


# SVM

In [14]:
from sklearn.svm import SVC

svc_params = {
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'degree':range(2,5)
}

svc = SVC()
clf = GridSearchCV(svc, svc_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8822410147991546
Best Parameters {'degree': 2, 'kernel': 'linear'}


# Ada-Boosting

In [15]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(x_scaled, y)

print('Train Accuracy Score', ada.score(x_scaled, y))

Train Accuracy Score 0.8521939953810623


# LSTM

In [ ]:
x_scaled = x_scaled[:,:,None]

In [ ]:
from keras import Sequential
from keras.layers import LSTM

model_lstm = Sequential()

model_lstm.add(LSTM(64, input_shape = x_scaled.shape[1:], return_sequences = True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.8))

model_lstm.add(LSTM(32, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(16, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(8, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(1, return_sequences=True, name='output'))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(Dense(64, activation = 'relu'))
model_lstm.add(Flatten())
model_lstm.add(Dense(1, activation='softmax'))

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model_lstm.summary()

2022-12-27 14:13:43.797353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 11720, 64)         16896     
                                                                 
 dropout (Dropout)           (None, 11720, 64)         0         
                                                                 
 lstm_1 (LSTM)               (None, 11720, 32)         12416     
                                                                 
 dropout_1 (Dropout)         (None, 11720, 32)         0         
                                                                 
 lstm_2 (LSTM)               (None, 11720, 16)         3136      
                                                                 
 dropout_2 (Dropout)         (None, 11720, 16)         0         
                                                                 
 lstm_3 (LSTM)               (None, 11720, 8)          8

In [ ]:
batch_size = 32

model_lstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)

Epoch 1/5
4/7 [================>.............] - ETA: 2:36 - loss: 0.6169 - accuracy: 0.0859

# CNN LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM

model_clstm  =  Sequential()

model_clstm.add(Conv1D(32, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(Conv1D(16, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(Conv1D(8, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(LSTM(32, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(LSTM(16, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(LSTM(1, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(Dense(64, activation = 'relu'))
model_clstm.add(Flatten())
model_clstm.add(Dense(1, activation = 'softmax'))

model_clstm.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model_clstm.summary()

In [ ]:
batch_size = 32

model_clstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)

# Bi-LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM, Embedding, Bidirectional


model_bilstm  =  Sequential()

model_bilstm.add(Bidirectional(LSTM(32, input_shape = x_scaled.shape[1:], return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Bidirectional(LSTM(16, return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Bidirectional(LSTM(1, return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Dense(64, activation = 'relu'))
model_bilstm.add(Flatten())
model_bilstm.add(Dense(1, activation = 'softmax'))

model_bilstm.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model_bilstm.summary()

In [ ]:
batch_size = 32

model_bilstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)